In [1]:
import os
from constants import *

os.environ["PPLX_API_KEY"] = PPLX_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [55]:
from constants import *
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import OpenAIModerationChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.document import Document
from langchain import hub
import asyncio


import pandas as pd
import numpy as np

In [3]:
import re
path = "data/reddit_opinion_climate_change.csv" # June 12
df = pd.read_csv(path)
df

,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,...,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
0,l8ai9g6,1,A few other things that are left out of the de...,climatechange,2024-06-12 16:36:53,1de0acd,Melodic-Hippo5536,0,1,0,...,71.0,1478.0,1549.0,97,NaN,New report refutes 33 false claims about solar...,0.99,97,0,2024-06-12 06:50:30
1,l8ai96k,1,So your take is not that carbon pricing doesn'...,canada,2024-06-12 16:36:50,1ddqrmw,The_Eternal_Void,0,1,0,...,1579.0,15065.0,16972.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
2,l8ai0r3,1,"Yes, except it's averaging higher.",canada,2024-06-12 16:35:32,1ddqrmw,LeeStrange,0,1,0,...,504.0,5127.0,5646.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
3,l8ai09f,1,"True, it's actually been way more devastating ...",canada,2024-06-12 16:35:28,1ddqrmw,Best-Hotel-1984,0,1,0,...,9.0,4419.0,4428.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
4,l8ahv26,1,The rich looking to get richer. Go green means...,climateskeptics,2024-06-12 16:34:39,1deajz2,vacouple3,0,1,0,...,352.0,4928.0,5342.0,3,NaN,New Report Reveals Massive Scale of Green Bill...,1.00,3,0,2024-06-12 16:15:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542498,dzyrmqu,2,"Yes, I remember the hype that ended in nothing...",climate_science,2018-06-01 22:18:21,8ntlug,MissPinga,0,2,0,...,301.0,3516.0,3817.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542499,dzyf2d7,2,There were a [lot of people who lost a lot of ...,climate_science,2018-06-01 18:58:56,8ntlug,silence7,0,2,0,...,982947.0,249288.0,1256803.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542500,dzyesw9,1,"Well, it can be used as a bio fuel. This is an...",climate_science,2018-06-01 18:55:08,8ntlug,dano1066,0,1,0,...,12834.0,184926.0,198366.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542501,dzybxzq,5,There's no money in algae,climate_science,2018-06-01 18:13:20,8ntlug,ZippymcOswald,0,5,0,...,39918.0,26950.0,67462.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16


In [4]:
df_deduplicated = df.drop_duplicates(subset='self_text', keep='first')
len(df_deduplicated)

533810

In [29]:
NUM_SAMPLES = 3000
working_series = df_deduplicated[df['subreddit']=='climate']
working_series = working_series.sample(NUM_SAMPLES, random_state=42)
working_series

/tmp/ipykernel_75611/4150199645.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  working_series = df_deduplicated[df['subreddit']=='climate']


,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,...,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
337451,kfzmufh,12,"Nothing in particular, just that I've noticed ...",climate,2024-01-02 15:56:17,18wpy55,blackcatwizard,0,12,0,...,1903.0,28509.0,30563.0,221,NaN,Hannah Ritchie: ‘Doomsday predictions are a dr...,0.87,221,0,2024-01-02 13:54:01
35169,l5xf5c3,12,Most people wanna shout about ‘morals!’ Until ...,climate,2024-05-27 20:02:32,1d1qnci,HumanityHasFailedUs,0,12,0,...,12.0,26108.0,26298.0,464,NaN,World has ‘moral responsibility’ to help small...,0.95,464,0,2024-05-27 12:43:00
24067,l6kgwb6,7,Man I hope Biden wins.,climate,2024-06-01 02:19:27,1d53m9f,Squibbles01,0,7,0,...,5613.0,80291.0,85978.0,3051,NaN,Project 2025 plans to dismantle the federal ag...,0.96,3051,0,2024-05-31 19:13:02
80851,l35so4q,4,"I like your positivity, hope is a wonderful th...",climate,2024-05-08 17:42:22,1cn9g38,mhicreachtain,0,4,0,...,18344.0,7153.0,25497.0,284,Truly The Age of Stupid,World’s top climate scientists expect global h...,0.98,284,0,2024-05-08 16:54:02
273721,kofquxt,1,The stock holders are not privy to the interna...,climate,2024-02-01 14:19:28,1aetib4,LudovicoSpecs,0,1,0,...,455464.0,612317.0,1088201.0,832,NaN,‘Smoking gun proof’: fossil fuel industry knew...,0.98,832,0,2024-01-30 17:10:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436333,ka09aa8,10,It’s incredible to see how this year continues...,climate,2023-11-20 11:57:38,17zewag,Villager723,0,10,0,...,1647.0,75335.0,77463.0,44,NaN,Global temperature for 17th November was the f...,0.99,44,0,2023-11-20 03:13:08
195951,kus2126,1,Why do you imagine I'm doing it ? \n\nWhat do ...,climate,2024-03-14 02:35:30,1bda6pu,CO2_3M_Year_Peak,0,1,0,...,22751.0,11289.0,34128.0,242,NaN,"Petition to Change the Name of ""Climate Change...",0.81,242,0,2024-03-12 22:08:34
404692,kbui5dk,2,I've just created a sub for the likes of Jaber...,climate,2023-12-03 18:48:20,189r9yd,fungussa,0,2,0,...,241812.0,68976.0,328652.0,1420,NaN,Cop28 president says there is ‘no science’ beh...,0.96,1420,0,2023-12-03 11:02:07
23419,l6nclqe,9,"They knew the truth 40 years ago and lied, pro...",climate,2024-06-01 16:59:59,1d54kj5,Kossimer,0,9,0,...,377.0,133781.0,134727.0,866,NaN,Vermont becomes the first US state to pass a l...,0.99,866,0,2024-05-31 19:54:48


In [6]:
# Text Length
print("Max:", np.max(working_series['self_text'].apply(len)))
print("Mean:", np.mean(working_series['self_text'].apply(len)))

Max: 6815
Mean: 238.76233333333334


In [7]:
def clean_text(text):
    # keep handles for model
    #text = re.sub("@\w+", "", text) # Remove handle   

    text = re.sub("[^\x20-\x7E]", "", text)
    text = re.sub('http\S+', "", text)

    return text

In [8]:
pplx_llm = ChatPerplexity(temperature=0, model="llama-3-70b-instruct")
openai_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
pplx_small_llm = ChatPerplexity(temperature=0, model="llama-3-8b-instruct")

In [52]:
import re 

def list_from_content(content, mstring):
    pattern = f"{mstring}(.*?){mstring}"
    match = re.search(pattern, content, re.DOTALL)

    if match:
        opinions_text = match.group().strip(mstring)
        opinions_text = opinions_text.strip("\n")

        opinions_list = [t.strip("\n") for t in opinions_text.split('- ')]
        opinions_list.pop(0)  # empty string
        
        return opinions_list
    else:
        raise Exception("Invalid LLM output!")


async def extract_statements(text_input, context, agent, mstring, max_attempts=3):
    
    opinion_list = []
    for i in range(max_attempts):
        try:
            content = agent.invoke({'input': text_input, "context": context, "mstring": mstring}).content
            opinion_list = list_from_content(content, mstring)
            break
        except Exception:
            continue
    
    return opinion_list


seg_human = """
Here are examples of segmentations.

Context: Climate Change.
Text Input: A disturbing trend on several fronts is the proliferation of e-bikes and e-scooters in contrast with purely human-powered vehicles.  Not only are the riders more sedentary but the growth in battery consumption is accelerating at an alarming rate.
Output: 

{mstring}
- Observes that traditional human-powered transportation is being replaced by electric vehicles.
- Observes that electric vehicles creates sedentary riders, implied to be bad.
- Deduces that an increase in electric vehicles accelerates battery consumption.
{mstring}

Can you extract opinions from the following input?

Context: {context}
Text Input: {input}
"""

system = """
You are a neutral, unbiased opinion extraction bot. 
Identify central opinions and statements in the text input and return them in a list. Each item should be specific and informative, while not relying on other items.
"""

seg_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", seg_human)])    



In [70]:
example = pd.DataFrame(working_series).sample(50, random_state=42)

full_text = example['post_title'] + "\n\n" + example['self_text']
full_text = full_text.to_list()
full_context = "Subreddit: " + example["subreddit"]
full_context = full_context.to_list()

agent = seg_prompt | pplx_small_llm
mstring = "###"

extraction = await asyncio.gather(
    *[extract_statements(t, c, agent, mstring) for t,c in zip(full_text, full_context)]
)

In [71]:
docs = [d for ds in extraction for d in ds]

print("Length:", len(docs))
docs

Length: 199


['States that the collapse of the biosphere will make economic troubles seem insignificant, implying that environmental degradation is a more pressing concern.',
 'Emphasizes that climate breakdown is the biggest problem, especially when considering the long-term life of a potential child, implying a sense of urgency and severity.',
 'Criticizes the idea of "baby steps" as insufficient to address climate change.',
 'Advocates for more drastic action, implying that "baby steps" are inadequate.',
 'Supports the idea of the U.S. taking steps to reduce climate change.',
 'States that climate change cannot be reversed, implying a pessimistic view.',
 'Expresses hope that the current situation is a one-off, implying a desire for a better outcome.',
 'Notes the rapid increase in global temperature due to CO2 levels, implying a sense of urgency and concern.',
 'Mentions the potential for 2C warming in 10 years, implying a sense of alarm and urgency.',
 'Observes that it is challenging to get C

In [74]:
import pickle
with open("example_docs", "wb") as fp:
   pickle.dump(docs, fp)

In [77]:
with open("example_docs", "rb") as fp:
   docs = pickle.load(fp)

In [86]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
doc_embed = await asyncio.gather(
    *[embeddings.embed_query(d) for d in docs] 
)

In [ ]:
from sklearn.manifold import TSNE



In [80]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(
    min_cluster_size=15,

)

topic_model = BERTopic(embedding_model=embeddings, hdbscan_model=hdbscan_model)
topics, probs = topic_model.fit_transform(docs)


ModuleNotFoundError: No module named 'cuml'

In [79]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,122,-1_the_that_to_and,"[the, that, to, and, of, is, climate, implying...",[Expresses frustration and cynicism towards th...
1,0,45,0_that_the_climate_is,"[that, the, climate, is, of, to, are, and, cha...",[Suggests that a solution to climate change mi...
2,1,16,1_and_home_electrification_green,"[and, home, electrification, green, expresses,...",[Expresses optimism about the potential of ele...
3,2,16,2_climate_trump_not_the,"[climate, trump, not, the, activists, democrat...",[Implies that demonizing Democrats may not be ...
